# Verifying the Projection of Moments with Signal MC
As found [previously](../moment-projection/moment_projection.ipynb), the projected moments have some missing factors causing them not to match expectations. A direct fit to data with moments as the parameters is now available for use, and so we can use those as "truth" information to compare our projected moments to. The fitted moments do have issues in obtaining moments with $>1\%$ contribution, but having ~8 moments should be enough to track down the missing factors.

This study will proceed as follows:
1. Generate Signal Monte Carlo (MC) according to a pseudo-realistic set of waves (no acceptance effects i.e. *thrown*)
   1. 35% polarization and in the PARA_0 orientation
2. Fit MC with same waveset, and obtain a fit result that should match the generated wave values
3. Project moments from the fit result to obtain a projected moment-set $H_{\text{proj}}$
4. Fit MC with the same number of moments, and obtain a fitted moment-set $H_{\text{fit}}$
5. Compare the fit and projected sets to investigate the missing factors.

## Setup

In [ ]:
# load libraries
import pandas as pd
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import pathlib

# load in the workspace dir so we can use package modules
current_dir = pathlib.Path.cwd()
WORKSPACE_DIR = None
while current_dir != current_dir.parent:
    if (current_dir / "neutralb1").is_dir():
        WORKSPACE_DIR = str(current_dir / "neutralb1")
        sys.path.insert(0, WORKSPACE_DIR)
    current_dir = current_dir.parent
if WORKSPACE_DIR is None:
    raise FileNotFoundError("Could not find 'neutralb1' directory in the path.")

import src.utils.utils as utils
import src.analysis.pwa_tools as pwa_tools

utils.load_shell_environment()
plt.style.use(f"{WORKSPACE_DIR}/config/pwa_plotter.mplstyle")

### Generate MC and fit with amplitudes
We'll use the same cfg file to generate and fit the amplitude-based Monte Carlo with. This will be done in a single bin of mass at:
* $0.1 < -t < 0.2$
* $8.2 < E_\gamma < 8.8$
* $1.20 < M_{\omega\pi^0} < 1.22$
  
Since by default the environment is setup to run on GPU, we will have to perform them separate from here in an interactive node session to produce the files. The process, run outside of this notebook, is:
1. `salloc --gres gpu:A100:1 --partition gpu --nodes 1 --mem-per-cpu=4G`
2. `bash run.sh`